In [3]:
import requests
import json

# Set up your Gemini API key here
api_key = ""



# Define your prompt formats
prompts = {
    'plain': 'Rewrite the following prompt using plain language, without changing its original meaning and structure. Only answer with the rephrased prompt.'
}

rephrased_questions = []



In [4]:
import google.generativeai as genai
from google.colab import userdata


genai.configure(api_key=api_key)

# Initialize the Generative Model
model = genai.GenerativeModel('gemini-1.5-flash-latest') # Replace with your desired model

In [5]:
from tqdm.notebook import tqdm

In [6]:
import re

def extract_option(text: str, option_number: int = 1) -> str:
    # Regex to capture the block for the requested option
    pattern = rf"\*\*Option {option_number}.*?(?=\*\*Option \d+|\Z)"
    match = re.search(pattern, text, re.S)
    if not match:
        return ""

    block = match.group()

    # Remove the "**Option N ..." header line
    block = re.sub(r"^\*\*Option.*?\n", "", block, count=1, flags=re.S)

    # Remove leading ">" characters and optional spaces from each line
    block = re.sub(r"^\s*>\s?", "", block, flags=re.M)

    # Remove all bold markers "**"
    block = block.replace("**", "")

    return block.strip()



In [ ]:
import os
import json
import re
from tqdm import tqdm
from pathlib import Path

folder = "results"
skip_prefixes = [
]

# Create output base directory if it doesn't exist
base_output_dir = Path("rephrased_prompts")
base_output_dir.mkdir(parents=True, exist_ok=True)

for filename in os.listdir(folder):
    if filename.endswith(".json") and not any(filename.startswith(p) for p in skip_prefixes):
        filepath = os.path.join(folder, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            data = json.load(f)

        prompt_data = data['all_results']
        rephrased_questions = []

        # Extract model name from filename
        model_match = re.search(r"task_(.*?)_(boomer|gen_z|gen_alpha|millennial|gen_x|academic|children|id)", filename)
        model_name = model_match.group(1) if model_match else "unknown_model"

        # Create subdirectory for the model
        model_output_dir = base_output_dir / model_name
        model_output_dir.mkdir(parents=True, exist_ok=True)

        for q in tqdm(prompt_data, desc=f"Rephrasing prompts for {filename}"):
            try:
                question_entry = {}
                for style, prompt_text in prompts.items():
                    full_prompt = f"{prompt_text}\n\nPrompt: \"{q['prompt_used']}\"\n\nRephrased prompt:"
                    response = model.generate_content([full_prompt])
                    question_entry[style] = response.text
                question_entry = {**question_entry, **q}
                rephrased_questions.append(question_entry)

            except Exception as e:
                print(f"\n{'*'*50} STOPPED {'*'*50}")
                partial_path = model_output_dir / f"rephrased_plain_{filename[:-5]}.json"
                with open(partial_path, "w", encoding="utf-8") as f:
                    json.dump(rephrased_questions, f, indent=4)
                raise e

        # Save full rephrased results
        output_path = model_output_dir / f"rephrased_plain_{filename[:-5]}.json"
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(rephrased_questions, f, indent=4)
        print(f"Rephrasing complete and results saved to '{output_path}'.")

        # Post-process to remove "**Option 1" and clean formatting
        no_options_qs = rephrased_questions.copy()
        for q in no_options_qs:
            if "**Option 1" in q['plain']:
                opt1 = extract_option(q['plain'], 1)
                if opt1:
                    q['plain'] = opt1
            q['plain'] = q['plain'].replace("**", "") + "\nAnswer:"

        one_option_path = model_output_dir / f"rephrased_plain_one_option_{filename[:-5]}.json"
        with open(one_option_path, "w", encoding="utf-8") as f:
            json.dump(no_options_qs, f, indent=4)
        print(f"Cleaned prompts saved to '{one_option_path}'.")
